# Import Data
In order to train our model, we are going to import time series data of room temperature.

With our RNN, we are going to try to forecast the temperature of a room given a certain air flow temperature.

You can find this dataset here: https://www.kaggle.com/datasets/vitthalmadane/ts-temp-1/data?select=MLTempDataset.csv

In [1]:
import kagglehub

# Download latest version of dataset to local filesystem
path = kagglehub.dataset_download("vitthalmadane/ts-temp-1")

print("Path to dataset files:", path)

Path to dataset files: /Users/noahcampise/.cache/kagglehub/datasets/vitthalmadane/ts-temp-1/versions/2


## Read Data into Memory
For this workshop, we are going to use pandas to read the data into memory and convert it into a numpy array to train our model

In [2]:
import pandas as pd
import os
import numpy as np

file_path = os.path.join(path, 'MLTempDataset.csv')
temperature_data = pd.read_csv(file_path)

In [3]:
temperature_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 6676 entries, 0 to 6675
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  6676 non-null   int64  
 1   Datetime1   6676 non-null   int64  
 2   DAYTON_MW   6676 non-null   float64
 3   Datetime    6676 non-null   str    
dtypes: float64(1), int64(2), str(1)
memory usage: 208.8 KB


## Data Visualization

In order to train our model, we need to understand what we are training the model on.

If we look at the top 5 rows of our temperature data, we are given 4 columns:

1. Unnamed: This represents the index of the data, since our data is in order, we can ignore this.
2. Datetime1: This is the hour of the day between 0 and 23
3. DAYTON_MW: This is the temperature of the room in Celsius
4. Datetime: This is the Date and hour of the day in a Datetime Format

We want our model to forecast temperature over time. So, we will treat our temperature as our label and we will Datetime as the time step.

In [4]:
temperature_data.head()

,Unnamed: 0,Datetime1,DAYTON_MW,Datetime
0,0,0,20.867,2022-01-04 00:00:00
1,1,1,21.000,2022-01-04 01:00:00
2,2,2,20.867,2022-01-04 02:00:00
3,3,3,20.650,2022-01-04 03:00:00
4,4,4,20.400,2022-01-04 04:00:00


# Data Preprocessing
Continuing with the data aspect of machine learning, we need to preprocess our data in a way that facilitates training for time series data.

As part of this, we will need to standardize our data.

In [5]:
# Convert pandas data to numpy for training
temperature_data = temperature_data.to_numpy()
temperature_data

array([[0, 0, 20.867, '2022-01-04 00:00:00'],
       [1, 1, 21.0, '2022-01-04 01:00:00'],
       [2, 2, 20.867, '2022-01-04 02:00:00'],
       ...,
       [6673, 21, 26.45, '2022-10-09 01:00:00'],
       [6674, 22, 25.9, '2022-10-09 02:00:00'],
       [6675, 23, 25.567, '2022-10-09 03:00:00']],
      shape=(6676, 4), dtype=object)

In [6]:
# Create function to create dataset
# Basically, we are saying, take 60 sequential temperatures, then, given this information
# predict what the next temperature would be.
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 2])
        y.append(data[i + time_step, 2])
    return np.array(X).astype('float64'), np.array(y).astype('float64')

# X, y = create_dataset(temperature_data)
# X = X.reshape(X.shape[0], X.shape[1], 1)


In [7]:
X, y = create_dataset(temperature_data)

In [8]:
X = X.reshape(X.shape[0], X.shape[1], 1)

In [9]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the Model

Using Tensorflow and Keras, we are going to build a Recurrent Nueral Network to forecast room temperature. In particular, we are going to use a Gated Recurrent Unit Model. 

At a high level, this addresses the vanishing gradient problem, allowing them to reliably predict over longer periods of time.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, GRU
model = Sequential([
    # GRU(64, activation='tanh', return_sequences=True, input_shape=(10, 5)),  # First GRU layer
    GRU(64, activation='tanh'),  # Second GRU layer
    Dense(1)  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='mse', metrics=['r2_score'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model
Now, we are going to train our model to fit our data and hopefully get a descent temperature forecaster

In [11]:
model.fit(X_train, y_train, epochs=20, batch_size=64)

predictions = model.predict(X_test)

Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 306.6031 - r2_score: -4.8947
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 114.4958 - r2_score: -1.2013
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 67.6649 - r2_score: -0.3009
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 39.2678 - r2_score: 0.2450
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 25.3228 - r2_score: 0.5131
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 18.3248 - r2_score: 0.6477
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 14.5871 - r2_score: 0.7195
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 12.4775 - r2_score: 0.7601
Epoch 9/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 11.2716 - r2_score: 0.7833
Epoch 10/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 10.4332 - r2_score: 0.7994
Epoch 11/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 9.8843 - r2_score: 0.8100
Epoch 12/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/s

In [12]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

42/42 - 0s - 5ms/step - loss: 2.5516 - r2_score: 0.8794

Test accuracy: 0.87938392162323


In [13]:
model.predict(X_train, verbose=0)[:10]

array([[23.87537 ],
       [25.065208],
       [25.323307],
       [25.668158],
       [25.916805],
       [11.763302],
       [10.515472],
       [11.079081],
       [17.061882],
       [12.486538]], dtype=float32)

In [14]:
y_train[:10].transpose()

array([24.867, 25.5  , 26.   , 26.267,  9.633,  9.267,  8.85 , 15.6  ,
       12.267,  9.067])

## Exporting the model
At this point, we have a satisfactory model. At this point, we are going to export this model to run.

In [23]:
!mkdir models
model.export('models/my_model.pb')

mkdir: models: File exists
INFO:tensorflow:Assets written to: models/my_model.pb/assets


INFO:tensorflow:Assets written to: models/my_model.pb/assets


Saved artifact at 'models/my_model.pb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  4722221008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4722219088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4722219472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4722218704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4722221776: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Optimization
There are techniques that we can use to reduce the size of Nueral Networks to reduce their size
and their computational cost.

Some of these include:
1. Weight Pruning
2. Weight Clustering
3. Quantization

For this workshop, we will talk about all three, but only implement quantization for our model.

# Weight Pruning

Magnitude-based weight pruning gradually zeroes out model weights during the training process to achieve model sparsity. Sparse models are easier to compress, and we can skip the zeroes during inference for latency improvements.

# Weight Clustering

Magnitude-based weight pruning gradually zeroes out model weights during the training process to achieve model sparsity. Sparse models are easier to compress, and we can skip the zeroes during inference for latency improvements.

## Quantization
Quantization aware training emulates inference-time quantization, creating a model that downstream tools will use to produce actually quantized models. The quantized models use lower-precision (e.g. 8-bit instead of 32-bit float), leading to benefits during deployment.

### Deploy with quantization

Quantization brings improvements via model compression and latency reduction. With the API defaults, the model size shrinks by 4x, and we typically see between 1.5 - 4x improvements in CPU latency in the tested backends. Eventually, latency improvements can be seen on compatible machine learning accelerators, such as the EdgeTPU and NNAPI.

In our workshop, we are going to use Post-training quantization, which will allow us to reduce model size with our already trained model.

In [29]:
import tensorflow as tf
# Here is how to specify 8 bit integer weight quantization
converter = tf.lite.TFLiteConverter.from_saved_model('models/my_model.pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_quant_model = converter.convert()
open("models/converted_model.tflite", "wb").write(tflite_quant_model)

W0000 00:00:1769574384.694122  175412 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1769574384.694148  175412 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2026-01-27 23:26:24.694347: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/my_model.pb
2026-01-27 23:26:24.694694: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2026-01-27 23:26:24.694698: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: models/my_model.pb
2026-01-27 23:26:24.697722: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2026-01-27 23:26:24.709637: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: models/my_model.pb
2026-01-27 23:26:24.715008: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 20666 microseconds.
2026-01-27 23:26:24.756750: W tensorflo

28824

In [30]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_quant_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '3' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the SHAPE op takes
tensor #0 as input and produces tensor #15 as output.

Subgraph#0 main(T#0) -> [T#28]
  Op#0 SHAPE(T#0) -> [T#15]
  Op#1 FlexTensorListReserve(T#13[-1, 64], T#12[1]) -> [T#16]
  Op#2 STRIDED_SLICE(T#15, T#11[0], T#10[1], T#10[1]) -> [T#17]
  Op#3 TRANSPOSE(T#0, T#9[1, 0, 2]) -> [T#18]
  Op#4 PACK(T#17, T#7[64]) -> [T#19]
  Op#5 FILL(T#19, T#8) -> [T#20]
  Op#6 WHILE(T#14[0], T#14[0], T#16, T#20, T#18, Cond: Subgraph#1, Body: Subgraph#2) -> [T#21, T#22, T#23, T#24, T#25]
  Op#7 FlexTensorListStack(T#23, T#13[-1, 64]) -> [T#26]
  Op#8 STRIDED_SLICE(T#26, T#4[-1, 0, 0], T#3[0, 0, 64], T#2[1, 1, 1]) -> [T#27]
  Op#9 FULLY_CONNECTED(T#27, T#5, T#1) -> [T#28]

Tensors of Subgraph#0
  T#0(serving_default_keras_tensor:0) shape_signature:[-1, 60, 1], type:FLOAT32
  T#1(arith.constant) shape:[1], type:FLOAT32 RO 4

In [31]:
tflite_interpreter = tf.lite.Interpreter(model_path="models/converted_model.tflite")

/Users/noahcampise/school-spring-2026/keras-workshop/.venv/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [32]:
tflite_interpreter.get_input_details()

[{'name': 'serving_default_keras_tensor:0',
  'index': 0,
  'shape': array([ 1, 60,  1], dtype=int32),
  'shape_signature': array([-1, 60,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [33]:
tflite_interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall_1:0',
  'index': 28,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [1]:
# I was not able to test the model. Weird version requirements.
input_index = tflite_interpreter.get_input_details()[0]["index"]
output_index = tflite_interpreter.get_output_details()[0]["index"]

test_temperatures = np.expand_dims(X_train[0], axis=0).astype(np.float32)

tflite_interpreter.allocate_tensors()

tflite_interpreter.set_tensor(input_index, test_temperatures)
tflite_interpreter.invoke()
predictions = tflite_interpreter.get_tensor(output_index)

NameError: name 'tflite_interpreter' is not defined